In [1]:
import numpy as np
import os,cv2,math
import skimage.io
from skimage.morphology import skeletonize
from skimage.filters import threshold_otsu
from skimage.morphology import opening,disk,closing
from keras_unet.models import custom_unet

Matplotlib is building the font cache; this may take a moment.
Using TensorFlow backend.


In [2]:
s = 256
input_shape = (s,s,3)

model = custom_unet(
    input_shape,
    num_layers=3,
    use_batch_norm=True,
    num_classes=1,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid')

model.load_weights('tap_root_dector_wts.h5')












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [3]:
directory='../sample_outputs/tap_roots'
if not os.path.exists(directory):
    os.makedirs(directory)

data_location = '../sample_images'
imagefile=os.listdir(data_location)
 
patchsize = 256

In [6]:
for file in imagefile:  
    image_path =data_location +'/' + file
    imagename=file.split('/')[-1]
    imagefull = cv2.imread(image_path)
    imagefull = cv2.cvtColor(imagefull, cv2.COLOR_BGR2RGB)
    h=math.ceil(imagefull.shape[0]/patchsize)*patchsize-imagefull.shape[0]
    w=math.ceil(imagefull.shape[1]/patchsize)*patchsize-imagefull.shape[1]
    original= cv2.copyMakeBorder(imagefull,0,h,0,w,cv2.BORDER_CONSTANT,value=[0,0,0])
    output_shape = (original.shape[0],original.shape[1])
    
    x_infer = np.array(cv2.resize(original,(s,s)))
    x_infer = np.asarray(x_infer, dtype=np.float32)/255
    x_infer = x_infer.reshape(1,x_infer.shape[0], x_infer.shape[1],x_infer.shape[2])
    y_pred_infer = model.predict(x_infer)
    
    image = y_pred_infer[0][:,:,0]
    thresh = threshold_otsu(image)
    selem = disk(1)
    binary = image > thresh
    #binary = opening(binary_p, selem)
    #binary = closing(binary, selem)
    
    binary = np.multiply(binary,255)
    binary = skimage.transform.resize(binary, output_shape=output_shape, mode='constant', preserve_range=True)
    cv2.imwrite(directory+'/'+imagename[:-4]+'.png',binary)